In [3]:
# import required libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
import matplotlib_inline
from matplotlib import cm
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [41]:
# extract all match names in directory as a list 
files = os.listdir('matches_2021_json')

In [42]:
 with open('matches_2021_json\\' + files[0], 'r') as f:
        data = json.load(f)

In [121]:
# avoid last file 'players.json' indexed as '-1'
cols = []
# find all column names in all json files
for i in files[:-1]:
    with open('matches_2021_json\\' + i, 'r') as f:
        data = json.load(f)
   
    # convert list of dictionaries into data frame 
    events = pd.DataFrame(data['events'])
    # reset indices
    events = events.reset_index()
    events.drop('index', axis = 1, inplace = True)
    cols = cols + list(events.columns)

In [128]:
# find all unique column names available
set(cols)

{'action',
 'attack_type',
 'corner_outcome',
 'end_time',
 'external_id',
 'foot_used',
 'game_time',
 'goal_mouth',
 'header',
 'next_player',
 'one_touch',
 'penalty',
 'player',
 'start_time',
 'team',
 'throw_in',
 'xdest',
 'xg',
 'xp',
 'xpos',
 'xt',
 'ydest',
 'ypos'}

In [87]:
col_rearr = ['external_id', 'team', 'player', 'next_player', 'xpos', 'ypos', 'xdest', 'ydest', 'start_time', 'end_time',
            'action', 'xg', 'xt', 'xp','game_time', 'header', 'penalty', 'attack_type', 
                    'corner_outcome', 'foot_used', 'one_touch', 'goal_mouth']

In [132]:
# avoid last file 'players.json' indexed as '-1'
for i in files[:-1]:
    with open('matches_2021_json\\' + i, 'r') as f:
        data = json.load(f)
   
    # convert list of dictionaries into data frame 
    events = pd.DataFrame(data['events'])
    
    # reset indices
    events = events.reset_index()
    events.drop('index', axis = 1, inplace = True)   
    
    # rearrange columns
    # since only single match has less than 23 columns which is '397511822166547-Halmstad-DegerforsIF'
    if len(events.columns) < 23:
        col_rearr = ['external_id', 'team', 'player', 'next_player', 'xpos', 'ypos', 'xdest', 'ydest', 'start_time', 'end_time',
                    'action', 'xg', 'xt', 'xp','game_time', 'header', 'penalty', 'throw_in']
    else:
        col_rearr = ['external_id','team', 'player', 'next_player', 'xpos', 'ypos', 'xdest', 'ydest', 'start_time', 
                     'end_time','action', 'xg', 'xt', 'xp','game_time', 'header', 'penalty', 'throw_in', 'attack_type', 
                    'corner_outcome', 'foot_used', 'one_touch', 'goal_mouth']
    events = events[col_rearr]
    
    # replace all None values with NaN (missing value)
    events.fillna(value = np.nan, inplace = True)
    
    # save dataframe 
    file_name = i.split('.')[0]
    events.to_csv('matches_2021_csv\\' + file_name + '.csv', index = False)

In [259]:
# load info 'how much players played in each game'
with open('matches_2021_json\\allsvenskan, 2021-players.json', 'r') as f:
    data2 = json.load(f)

# aggregate results into data frame
playing_time = pd.DataFrame()
for i in data2:
    single_col = {'player_name':i['name'], 'team_name':i['team'],
                  'minutes_played':sum(i['minutes']['62163887d0736a3c7964ff4b'].values())}
    playing_time = pd.concat([playing_time, pd.DataFrame([single_col])])

playing_time.reset_index(inplace = True, drop = True)

In [260]:
playing_time

,player_name,team_name,minutes_played
0,Christopher Mc Vey,IF Elfsborg,2307
1,Tashreeq Matthews,Varbergs BoIS FC,1969
2,Ali Youssef,BK Häcken,389
3,Mikael Lustig,AIK,2491
4,Davor Blazevic,Hammarby,96
...,...,...,...
432,Emanuel Chabo,IFK Norrköping FK,212
433,Johannes Bjarnason,IFK Norrköping FK,198
434,Kalpi Wilfried Ouattara,Östersund,92
435,Sebastian Lagerlund,BK Häcken,200
